In [1]:
import pandas as pd
import re

def parse_file(path, name):
    data = []
    current_test = {}  # Armazenará os dados completos de um teste (BB + Aprox)
    last_method = None
    
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()

            # Dados do problema
            if line.startswith('* Número de itens: '):
                # Se já temos dados do método anterior, prepara para novo teste
                if last_method == '2-aproximativo-guloso':
                    data.append(current_test)
                    current_test = {}
                current_test['N'] = line.split(': ')[1].strip()
            elif line.startswith('* Peso médio:'):
                current_test['w_avg'] = line.split(': ')[1].strip()
            elif line.startswith('* Peso máximo:'):
                current_test['w_max'] = line.split(': ')[1].strip()
            elif line.startswith('* Valor médio:'):
                current_test['v_avg'] = line.split(': ')[1].strip()
            elif line.startswith('* Valor máximo:'):
                current_test['v_max'] = line.split(': ')[1].strip()
            elif line.startswith('* Capacidade da Mochila:'):
                current_test['W'] = line.split(': ')[1].strip()

            # Branch and Bound
            elif line.startswith('Branch and Bound:'):
                current_test['BB_sol'] = line.split(': ')[1].strip()
                last_method = 'BB'
            elif line.startswith('Tempo Branch and Bound :'):
                current_test['BB_time'] = line.split(': ')[1].strip()
            elif line.startswith('* Memória residente real (RSS):') and '[BB]' in current_test.get('last_section', ''):
                current_test['BB_mem'] = line.split(': ')[1].replace(' KB', '').strip()

            # FPTAS
            elif line.startswith('2-aproximativo-fptas:'):
                current_test['FPTAS_sol'] = line.split(': ')[1].strip()
                last_method = '2-aproximativo-fptas'
            elif line.startswith('Tempo Aproximativo FPTAS:'):
                current_test['FPTAS_time'] = line.split(': ')[1].strip()
            elif line.startswith('* Memória residente real (RSS):') and '[fptas]' in current_test.get('last_section', ''):
                current_test['FPTAS_mem'] = line.split(': ')[1].replace(' KB', '').strip()
                current_test['type'] = name

             # Guloso
            elif line.startswith('2-aproximativo-guloso:'):
                current_test['greedy_sol'] = line.split(': ')[1].strip()
                last_method = '2-aproximativo-guloso'
            elif line.startswith('Tempo Aproximativo Guloso:'):
                current_test['greedy_time'] = line.split(': ')[1].strip()
            elif line.startswith('* Memória residente real (RSS):') and '[guloso]' in current_test.get('last_section', ''):
                current_test['greedy_mem'] = line.split(': ')[1].replace(' KB', '').strip()
                current_test['type'] = name
            
            # Track current section
            elif line.startswith('=== Métricas de Memória'):
                current_test['last_section'] = line
    
    # Add the last test if exists
    if current_test:
        data.append(current_test)
    
    return pd.DataFrame(data)

# Process both files
df_large = parse_file("large_results.txt", "Large")
df_low = parse_file("low_results.txt", "Low")

# Combine both DataFrames
df_combined = pd.concat([df_large, df_low], ignore_index=True)

# Convert scientific notation to normal numbers and clean data
def clean_value(val):
    if not isinstance(val, str):
        return val
    if 'e+' in val:
        return str(float(val))
    return val.replace(' KB', '')

df_combined = df_combined.applymap(clean_value)

# Reorder columns
columns = ['type', 'N', 'W','w_avg','w_max','v_avg','v_max','BB_sol', 'BB_time', 'BB_mem', 
           'FPTAS_sol', 'FPTAS_time', 'FPTAS_mem','greedy_sol', 'greedy_time', 'greedy_mem']
df_combined = df_combined[columns]

# Fill empty values with empty string for better CSV output
df_combined = df_combined.fillna('')

# Save to CSV
df_combined.to_csv('resultados_combinados.csv', index=False)

print("DataFrame consolidado:")
print(df_combined)

DataFrame consolidado:
     type    N      W    w_avg    w_max    v_avg    v_max   BB_sol BB_time  \
0   Large  200    997   498.09      997   711.59     1230     5397    1556   
1   Large  500   2543  513.864      997  494.142      998    28857  338508   
2   Large  500   2543  513.864      997  516.634     1074     4566   47974   
3   Large  500   2517  508.612      998  608.612     1098     7117  264675   
4     Low   20    879     54.9       96     54.3       91     1025    1997   
5     Low   10    269     53.9       95     41.2       87      295      27   
6     Low   20    878     54.9       96    54.25       91     1024    2792   
7     Low    4     20     6.75        9       12       15       35      12   
8     Low    4     11     4.75        7    10.25       13       23      14   
9     Low   15    375  49.4611  89.5962  37.5331  98.8525  481.069      60   
10    Low   10     60       13       30     10.5       20       52     151   
11    Low    7     50  13.2857       31  